# Arctic

> Arctic helper scripts and functions.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp arctic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, LibraryOptions
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import (
    MainConfig,
    Overrides,
    NASDAQExchange,
    ETFMembers,
    register_configs,
    get_config,
)
from lobster_tools.preprocessing import Data, Lobster, infer_ticker_to_date_range, infer_ticker_to_ticker_path, infer_ticker_dict
import sys
import pandas as pd
from logging import Logger
from datetime import date
from dataclasses import dataclass
import time

from concurrent.futures import ProcessPoolExecutor, wait
import subprocess

When using the `@hydra.main` decorator, `register_configs` must be called. If simply using a notebook or writing the CLIs with `click`, it is enough to use `get_config`.

In [ ]:
# | export
register_configs()
cfg = get_config(overrides=Overrides.full_server)

In [ ]:
# | export
CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
    show_default=True,
)

In [ ]:
# @click.group()
# @click.option('--debug/--no-debug', default=False)
# @click.pass_context
# def cli(ctx, debug):
#     # ensure that ctx.obj exists and is a dict (in case `cli()` is called
#     # by means other than the `if` block below)
#     ctx.ensure_object(dict)

#     ctx.obj['DEBUG'] = debug

# @cli.command()
# @click.pass_context
# def sync(ctx):
#     click.echo(f"Debug is {'on' if ctx.obj['DEBUG'] else 'off'}")

# if __name__ == '__main__':
#     cli(obj={})

In [ ]:

# option_dict = {
#     'db_path': click.option("-d", "--db_path", default=cfg.db.db_path, help="Database path"),
#     'library': click.option("-l", "--library", default=cfg.db.db_path, help="Library name"),
# }

# # Custom decorator to apply options based on a list of names
# def apply_options(option_names):
#     def decorator(f):
#         for option_name in reversed(option_names):
#             f = option_dict[option_name](f)
#         return f
#     return decorator

In [ ]:
# | export
class Options:
    def __init__(self) -> None:
        self.db_path = click.option("-d", "--db_path", default=cfg.db.db_path, help="Database path")
        self.library = click.option("-l", "--library", default=cfg.db.db_path, help="Library name")

def apply_options(options: list):
    def decorator(f):
        for option in reversed(options):
            f = option(f)
        return f
    return decorator

@click.group()
def arctic():
    pass

O = Options()
@arctic.command()
@apply_options([O.db_path])
def initdb(db_path):
    print(f'Initialized the database {db_path}')
    # click.echo(f'Initialized the database {db_path}')

@arctic.command()
@apply_options([O.db_path, O.library])
def use_both(db_path, library):
    print(f'Initialized the database {db_path} {library}')
    # click.echo(f'Initialized the database {db_path}')

@arctic.command()
def dropdb():
    click.echo('Dropped the database')

NameError: name 'click' is not defined

In [ ]:
# | export
click_db_path = click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
click_library = click.option("-l", "--library", default=cfg.db.library, help="library name")

@click.group()
def cool():
    pass

@cool.command()
@click_db_path
def initdb(db_path):
    print(f'Initialized the database {db_path}')
    # click.echo(f'Initialized the database {db_path}')

@cool.command()
def dropdb():
    click.echo('Dropped the database')

In [ ]:
# | export
@click.group()
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.pass_context
def nic(ctx, db_path, library):
    ctx.ensure_object(dict)
    ctx.obj['DB_PATH'] = db_path
    ctx.obj['LIBRARY'] = library

@nic.command()
@click.pass_context
def initdb(ctx):
    db_path = ctx.obj['DB_PATH']
    print(f'Initialized the database {db_path}')
    click.echo(f'Initialized the database {db_path}')

@nic.command()
@click.pass_context
def dropdb(ctx):
    click.echo('Dropped the database')

def nic_entrypoint():
    nic(obj={}, auto_envvar_prefix='NIC')

In [ ]:
# | export
def get_arctic_library(db_path, library):
    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic_library = arctic[library]
    return arctic_library

In [ ]:
# | export
# TODO: csv_path vs csv_files_path. think if this is a problem...maybe unify?
class Options:
    def __init__(self) -> None:
        self.db_path = click.option("-d", "--db_path", default=cfg.db.db_path, help="Database path")
        self.library = click.option("-l", "--library", default=cfg.db.library, help="Library name")
        self.ticker = click.option("-t", "--ticker", required=True, help="ticker to print")
        self.start_date = click.option("-s", "--start_date", default=None, help="start date")
        self.end_date = click.option("-e", "--end_date", default=None, help="end date")
        self.csv_path = click.option("-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path")
        self.etf = click.option("--etf", default=None, help="restrict to subset specified by ETF members")
        self.zip_path = click.option("-z", "--zip_path", default="/nfs/lobster_data/lobster_raw/2016", help="zip files path")
        self.max_workers = click.option("-m", "--max_workers", default=20, help="max workers for parallelisation")
O = Options()

def apply_options(options: list):
    def decorator(f):
        for option in reversed(options):
            f = option(f)
        return f
    return decorator

@click.group(context_settings=CONTEXT_SETTINGS)
def arctic():
    pass

@arctic.command()
@apply_options([O.db_path])
def list_libraries(db_path) -> None:
    """List arcticdb libraries"""
    arctic = Arctic(f"lmdb://{db_path}")
    print(arctic.list_libraries())

@arctic.command()
@apply_options([O.db_path, O.library])
def list_symbols(db_path, library) -> None:
    """List symbols in the arcticdb library."""
    arctic = Arctic(f"lmdb://{db_path}")
    print(arctic[library].list_symbols())

@arctic.command()
@apply_options([O.db_path, O.library])
def create_library(db_path, library) -> None:
    """Create a blank new arcticdb library."""
    arctic = Arctic(f"lmdb://{db_path}")
    arctic.create_library(library) 
    print(arctic[library])

@arctic.command()
@apply_options([O.db_path, O.library])
@click.confirmation_option(prompt='Are you sure you want to delete the entire library?')
def delete_library(db_path, library) -> None:
    """Delete entire arcticdb library"""
    arctic = Arctic(f"lmdb://{db_path}")
    arctic.delete_library(library) 

@arctic.command()
@apply_options([O.db_path, O.library, O.ticker, O.start_date, O.end_date])
def read(db_path, library, ticker, start_date, end_date,
):
    """Read ticker and print head and tail."""
    arctic = Arctic(f"lmdb://{db_path}")

    if start_date and end_date:
        start_datetime = pd.Timestamp(f"{start_date}T{NASDAQExchange.exchange_open}")
        end_datetime = pd.Timestamp(f"{end_date}T{NASDAQExchange.exchange_close}")
        date_range = (start_datetime, end_datetime)
        df = arctic[library].read(ticker, date_range=date_range).data
    else:
        df = arctic[library].read(ticker).data
    
    print(f"Printing df.head() and df.tail() for ticker {ticker}")
    print(df.head())
    print(df.tail())

def write_(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    """Preprocess and write ticker to database."""
    arctic = Arctic(f"lmdb://{db_path}")

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)
    print(f"head of ticker {ticker}")
    print(df.head())

    arctic[library].write(symbol=ticker, data=df)

@arctic.command()
@apply_options([O.db_path, O.library, O.csv_path, O.ticker, O.start_date, O.end_date])
def write(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    """Preprocess and write ticker to database."""
    write(
    db_path=db_path,
    library=library,
    csv_path=csv_path,
    ticker=ticker,
    start_date=start_date,
    end_date=end_date,
    )

@arctic.command()
@apply_options([O.db_path, O.library, O.csv_path, O.start_date, O.end_date])
def generate_jobs(db_path, library, csv_path, start_date, end_date):
    ticker_date_dict = infer_ticker_to_date_range(csv_path)
    with open('arctic_commands.txt', 'w') as f:
        for ticker, (inferred_start_date, inferred_end_date) in ticker_date_dict.items():
            # if date is None use the inferred date, otherwise use the CLI argument
            start_date = start_date or inferred_start_date
            end_date = end_date or inferred_end_date
            f.write(f"arctic write --csv_path={csv_path} --db_path={db_path} --library={library} --ticker={ticker} --start_date={start_date} --end_date={end_date} \n")

def sleepy(csv_path, folder_info):
    time.sleep(5)
    print(csv_path, folder_info.full)

def extract_7z(input_path, output_path):
    try:
        subprocess.run(["7z", "x", input_path, f"-o{output_path}"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {str(e)}")

@arctic.command()
@apply_options([O.zip_path, O.csv_path, O.etf, O.max_workers])
def zip(zip_path, csv_path, etf, max_workers):
    folder_infos = infer_ticker_dict(zip_path)

    # filter first
    if etf:
        def in_etf(folder_info):
            return folder_info.ticker in ETFMembers().mapping[etf] + [etf]
        folder_infos = list(filter(in_etf, folder_infos))

    # commands = [f"mkdir -p {csv_path}/{folder_info.ticker_till_end}\n"
    #             for folder_info in folder_infos]

    # with open("zip_commands.txt", "w") as f:
    #     [f.write(command) for command in commands]
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # outputs_dirs = [folder_info.ticker_till_end for folder_info in folder_infos]
        futures = [
            executor.submit(os.mkdir, path=f"{csv_path}/{folder_info.ticker_till_end}")
            for folder_info in folder_infos
        ]
        wait(futures)
        futures = [
            executor.submit(extract_7z, input_path=folder_info.full, output_path=f"{csv_path}/{folder_info.ticker_till_end}")
            for folder_info in folder_infos
        ]


        # for folder_info in folder_infos:
        #     # print(folder_info.ticker)
        #     f.write(f"examle: mkdir {csv_path}/{folder_info.ticker_till_end}\n")


    # ticker_date_dict = infer_ticker_to_ticker_path(zip_path)
    # print(ticker_date_dict)
    # if etf:
    #     print(ETFMembers().mapping[etf])
    #     ticker_date_dict = {
    #         ticker: ticker_path
    #         for ticker, ticker_path in ticker_date_dict.items()
    #         if ticker in ETFMembers().mapping[etf] + [etf]
    #     }
    # print(ticker_date_dict)
    # ticker_dict = infer_ticker_dict(zip_path)
    # with open("zip_commands.txt", "w") as f:
    #     for ticker, dict_ in ticker_dict.items():
    #         full = dict_["full"]
    #         ticker_till_end = dict_["ticker_till_end"]
    #         f.write(f"mkdir {csv_path}/{ticker_till_end}\n")
    #         f.write(f"/nfs/home/nicolasp/usr/bin/7z x {full} -o{ticker_till_end}\n")


@arctic.command()
@apply_options([O.db_path, O.library, O.csv_path, O.max_workers])
def dump(
    db_path,
    library,
    csv_path,
    max_workers,
):
    """Dump all csv to arctic_db inferring start and end date from folder."""
    folder_infos = infer_ticker_dict(csv_path)
    print(folder_infos)

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(write_, csv_path=csv_path, db_path=db_path, library=library, ticker=folder_info.ticker, start_date=folder_info.start_date, end_date=folder_info.end_date)
            for folder_info in folder_infos
        ]
        for future in futures:
            print(future.result())
    print('done')


In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_list_symbols(db_path, library) -> None:
    """List symbols in the arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)
    print(f"Symbols in library {library}")
    print(arctic_library.list_symbols())

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_create_new_library(db_path, library) -> None:
    """Create a blank new arcticdb library."""
    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic.create_library(library) 
    print(arctic[library])

In [ ]:

# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
def arctic_list_libraries(db_path) -> None:
    """List arcticdb libraries"""

    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    print(arctic.list_libraries())

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_delete_library(db_path, library) -> None:
    """Delete arcticdb library"""

    user_input = input("Proceed by deleting this entire library? (y/n): ")
    user_input = user_input.lower()
    match user_input:
        case "y":
            pass
        case "n":
            sys.exit(0)
        case _:
            sys.exit(1)

    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic.delete_library(library) 

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to print")
@click.option("-s", "--start_date", default=None, help="start date")
@click.option("-e", "--end_date", default=None, help="end date")
def arctic_read_symbol(db_path, library, ticker, start_date, end_date,
):
    """Print df.head() and available columns for ticker in arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    if start_date and end_date:
        start_datetime = pd.Timestamp(f"{start_date}T{NASDAQExchange.exchange_open}")
        end_datetime = pd.Timestamp(f"{end_date}T{NASDAQExchange.exchange_close}")
        date_range = (start_datetime, end_datetime)
        df = arctic_library.read(ticker, date_range=date_range).data
    else:
        df = arctic_library.read(ticker).data
    
    print(f"Printing df.head() and df.tail() for ticker {ticker}")
    print(df.head())
    print(df.tail())

Write to arctic again

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
def arctic_write_symbol(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    # if ticker in arctic_library.list_symbols():
    #     print("warning - there is already data for ths ticker")
    #     user_input = input("Proceed by adding data to this symbol? (y/n): ")
    #     user_input = user_input.lower()
    #     match user_input:
    #         case "y":
    #             pass
    #         case "n":
    #             sys.exit(0)
    #         case _:
    #             sys.exit(1)

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)

    arctic_library.write(symbol=ticker, data=df)

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-s", "--start_date", default=None, help="start date")
@click.option("-e", "--end_date", default=None, help="end date")
def arctic_generate_jobs(csv_path, db_path, library, start_date, end_date):
    ticker_date_dict = infer_ticker_to_date_range(csv_path)
    with open('arctic_commands.txt', 'w') as f:
        for ticker, (inferred_start_date, inferred_end_date) in ticker_date_dict.items():
            # if date is None use the inferred date, otherwise use the CLI argument
            start_date = start_date or inferred_start_date
            end_date = end_date or inferred_end_date
            f.write(f"arctic_write_symbol --csv_path={csv_path} --db_path={db_path} --library={library} --ticker={ticker} --start_date={start_date} --end_date={end_date} \n")

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-z",
    "--zip_path",
    default="/nfs/lobster_data/lobster_raw/2016",
    help="zip files path",
)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option(
    "-e", "--etf", default=None, help="restrict to subset specified by ETF members"
)
def zip_generate_jobs(zip_path, csv_path, etf):
    # ticker_date_dict = infer_ticker_to_ticker_path(zip_path)
    # print(ticker_date_dict)
    # if etf:
    #     print(ETFMembers().mapping[etf])
    #     ticker_date_dict = {
    #         ticker: ticker_path
    #         for ticker, ticker_path in ticker_date_dict.items()
    #         if ticker in ETFMembers().mapping[etf] + [etf]
    #     }
    # print(ticker_date_dict)
    ticker_dict = infer_ticker_dict(zip_path)
    with open("zip_commands.txt", "w") as f:
        for ticker, dict_ in ticker_dict.items():
            full = dict_["full"]
            ticker_till_end = dict_["ticker_till_end"]
            f.write(f"mkdir {csv_path}/{ticker_till_end}\n")
            f.write(f"/nfs/home/nicolasp/usr/bin/7z x {full} -o{ticker_till_end}\n")

In [ ]:
# | exports
# | code-fold: true
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
def arctic_dump_all(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    if ticker in arctic_library.list_symbols():
        print("warning - there is already data for ths ticker")
        user_input = input("Proceed by adding data to this symbol? (y/n): ")
        user_input = user_input.lower()
        match user_input:
            case "y":
                pass
            case "n":
                sys.exit(0)
            case _:
                sys.exit(1)

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)
    print(df)

    arctic_library.append(symbol=ticker, data=df)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()